# Load Libraries

In [2]:
# Load libraries
import numpy as np
import pandas as pd
import io

# Load Data

In [2]:
from google.colab import files
uploaded = files.upload()

Saving dataset_mock_final.csv to dataset_mock_final.csv


In [15]:
dat = pd.read_csv('../data/dataset_mock_final.csv', sep = ";")
dat

,date,severity,mortality_ratio,age,num_proc,ambulatory,origin,expected_length,tip_grd,tip_adm,exitus,dataset
0,2016-07,NaN,0.001193,15603.0,4.0,NaN,NaN,7.0,M,1.0,0,train
1,2016-05,1.0,0.000000,14285.0,3.0,NaN,1.0,NaN,M,1.0,0,train
2,2016-01,NaN,0.000000,6046.0,2.0,NaN,NaN,2.0,NaN,1.0,0,train
3,2016-01,1.0,0.004060,27340.0,4.0,NaN,2.0,9.0,Q,NaN,0,train
4,2016-05,2.0,0.028365,28685.0,10.0,0.0,NaN,9.0,M,1.0,0,train
...,...,...,...,...,...,...,...,...,...,...,...,...
10807,2016-12,2.0,0.028365,23619.0,2.0,NaN,NaN,2.0,NaN,1.0,0,test
10808,2016-12,1.0,0.000606,3935.0,1.0,NaN,1.0,2.0,M,1.0,0,test
10809,2016-12,NaN,0.040452,30163.0,4.0,NaN,NaN,2.0,M,NaN,0,test
10810,2016-12,NaN,0.000000,29012.0,4.0,NaN,NaN,0.0,NaN,1.0,0,test


Let's remove date variable, as it is not useful.

In [16]:
dat.drop('date', axis = 1, inplace = True)

# One Hot Encoding

In [17]:
from sklearn.preprocessing import OneHotEncoder

In [18]:
categorical_vars = ['severity', 'origin', 'ambulatory', 'tip_grd', 'tip_adm']
non_categorical_vars = list(set(dat.columns) - set(categorical_vars))

In [19]:
# 3) Define "model"
ohe = OneHotEncoder(sparse_output = False, drop = 'first')

# 4) Train "model"
ohe.fit(dat[categorical_vars][dat['dataset'] == 'train'])

# 5) "Predict"
dat_ohe = pd.DataFrame(ohe.transform(dat[categorical_vars]))

# Optional
dat_ohe.columns = ohe.get_feature_names_out()

# Combine numerical and categorical
dat = pd.concat((dat[non_categorical_vars], dat_ohe), axis=1)

In [8]:
dat

,mortality_ratio,expected_length,age,num_proc,exitus,dataset,severity_2.0,severity_3.0,severity_4.0,severity_nan,...,origin_8.0,origin_9.0,origin_nan,ambulatory_1.0,ambulatory_nan,tip_grd_Q,tip_grd_nan,tip_adm_2.0,tip_adm_3.0,tip_adm_nan
0,0.001193,7.0,15603.0,4.0,0,train,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,NaN,14285.0,3.0,0,train,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,2.0,6046.0,2.0,0,train,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,0.004060,9.0,27340.0,4.0,0,train,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
4,0.028365,9.0,28685.0,10.0,0,train,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10807,0.028365,2.0,23619.0,2.0,0,test,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
10808,0.000606,2.0,3935.0,1.0,0,test,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
10809,0.040452,2.0,30163.0,4.0,0,test,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
10810,0.000000,0.0,29012.0,4.0,0,test,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


# Preprocessing

**Perform the following preprocessing steps:**

**- Fill numerical missing values with MICE employing a Random Forest as model por imputation of missing values.**

In [20]:
from fancyimpute import IterativeImputer as MICE
from sklearn.ensemble import RandomForestRegressor

In [21]:
numerical_vars = list(set(dat.columns) - set(['exitus', 'dataset']))

In [22]:
# 3) Define "model"
model = MICE(RandomForestRegressor())

# 4) Train "model"
model.fit(dat[numerical_vars][dat['dataset'] == 'train'])

# 5) "Predict"
dat[numerical_vars] = model.transform(dat[numerical_vars])

J:\IE\SECOND TERM\MLII\venv\Lib\site-packages\sklearn\impute\_iterative.py:796: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [12]:
dat

,mortality_ratio,expected_length,age,num_proc,exitus,dataset,severity_2.0,severity_3.0,severity_4.0,severity_nan,...,origin_8.0,origin_9.0,origin_nan,ambulatory_1.0,ambulatory_nan,tip_grd_Q,tip_grd_nan,tip_adm_2.0,tip_adm_3.0,tip_adm_nan
0,0.001193,7.00,15603.0,4.0,0,train,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,5.56,14285.0,3.0,0,train,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,2.00,6046.0,2.0,0,train,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,0.004060,9.00,27340.0,4.0,0,train,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
4,0.028365,9.00,28685.0,10.0,0,train,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10807,0.028365,2.00,23619.0,2.0,0,test,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
10808,0.000606,2.00,3935.0,1.0,0,test,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
10809,0.040452,2.00,30163.0,4.0,0,test,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
10810,0.000000,0.00,29012.0,4.0,0,test,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [13]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10812 entries, 0 to 10811
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   mortality_ratio  10812 non-null  float64
 1   expected_length  10812 non-null  float64
 2   age              10812 non-null  float64
 3   num_proc         10812 non-null  float64
 4   exitus           10812 non-null  int64  
 5   dataset          10812 non-null  object 
 6   severity_2.0     10812 non-null  float64
 7   severity_3.0     10812 non-null  float64
 8   severity_4.0     10812 non-null  float64
 9   severity_nan     10812 non-null  float64
 10  origin_2.0       10812 non-null  float64
 11  origin_3.0       10812 non-null  float64
 12  origin_4.0       10812 non-null  float64
 13  origin_6.0       10812 non-null  float64
 14  origin_8.0       10812 non-null  float64
 15  origin_9.0       10812 non-null  float64
 16  origin_nan       10812 non-null  float64
 17  ambulatory_1

**-Use SMOTE to achieve a 10/90 ratio (exitus = 1/exitus = 0) on the train set**

In [23]:
from imblearn.over_sampling import SMOTE

In [24]:
sampling_strategy = 10/90

In [25]:
# Apply SMOTE
sm = SMOTE(sampling_strategy = sampling_strategy,
      random_state = 0,
      k_neighbors = 5)
X_res, y_res = sm.fit_resample(dat[dat['dataset'] == 'train'].drop(['dataset', 'exitus'], axis = 1), dat[dat['dataset'] == 'train']['exitus'])


# Add dataset and exitus columns
X_res['exitus'] = y_res
X_res['dataset'] = 'train'

# Concatenate new train set after SMOTE with original validation and test sets
dat = pd.concat([X_res, dat[dat['dataset'] == 'val'], dat[dat['dataset'] == 'test']])

J:\IE\SECOND TERM\MLII\venv\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "J:\IE\SECOND TERM\MLII\venv\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


In [26]:
100*dat[dat['dataset'] == 'train'].exitus.value_counts()/dat[dat['dataset'] == 'train'].shape[0]

exitus
0    90.009337
1     9.990663
Name: count, dtype: float64

# Models

**After carrying out this preprocessing, you should do a grid testing the following models:**

**- Random Forest**

In [18]:
from sklearn.ensemble import RandomForestClassifier as model_constructor
from sklearn.metrics import roc_auc_score as metric

In [19]:
# Random Forest
n_estimators_values = [10, 100, 1000];
max_features_values = [2, 5, 10]
max_samples_values = [100, 1000, dat[dat['dataset'] == 'train'].shape[0]]

params_grid = {'max_features': max_features_values,
              'n_estimators': n_estimators_values,
               'max_samples': max_samples_values}


In [20]:
num_iter = 1;
grid_results = pd.DataFrame(columns = ('max_features',
                                       'n_estimators',
                                       'max_samples',
                                       'metric_train',
                                       'metric_val'))

for max_features in params_grid['max_features']:
    for n_estimators in params_grid['n_estimators']:
        for max_samples in params_grid['max_samples']:

                        # Print trace
                        print('Iteracion = ' + str(num_iter))

                        # [3] Define model
                        model = model_constructor(max_features = max_features,
                                                  n_estimators = n_estimators,
                                                  max_samples = max_samples,
                                                  random_state = 0)

                        # [4] Train model
                        model.fit(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1),
                                  dat[dat['dataset'] == 'train'].exitus.values)


                        # [5] Predict
                        pred_train = model.predict_proba(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1)) # predict!
                        pred_val = model.predict_proba(dat[dat['dataset'] == 'val'].drop(['exitus', 'dataset'], axis = 1)) # predict!

                        # [6] Compute metric
                        metric_train = metric(dat[dat['dataset'] == 'train'].exitus.values, pred_train[:,1])
                        metric_val = metric(dat[dat['dataset'] == 'val'].exitus.values, pred_val[:,1])

                        # print error
                        print('Metric train = %.2f - Metric validation = %.2f.'
                              % (metric_train, metric_val))

                        # Save iteration results
                        grid_results.loc[num_iter]=[ max_features,
                                                    n_estimators,
                                                    max_samples,
                                                 metric_train,
                                                 metric_val]
                        num_iter += 1



Iteracion = 1
Metric train = 0.95 - Metric validation = 0.85.
Iteracion = 2
Metric train = 1.00 - Metric validation = 0.84.
Iteracion = 3
Metric train = 1.00 - Metric validation = 0.84.
Iteracion = 4
Metric train = 0.98 - Metric validation = 0.92.
Iteracion = 5
Metric train = 1.00 - Metric validation = 0.90.
Iteracion = 6
Metric train = 1.00 - Metric validation = 0.90.
Iteracion = 7
Metric train = 0.99 - Metric validation = 0.92.
Iteracion = 8
Metric train = 1.00 - Metric validation = 0.91.
Iteracion = 9
Metric train = 1.00 - Metric validation = 0.91.
Iteracion = 10
Metric train = 0.96 - Metric validation = 0.87.
Iteracion = 11
Metric train = 1.00 - Metric validation = 0.82.
Iteracion = 12
Metric train = 1.00 - Metric validation = 0.82.
Iteracion = 13
Metric train = 0.98 - Metric validation = 0.91.
Iteracion = 14
Metric train = 1.00 - Metric validation = 0.90.
Iteracion = 15
Metric train = 1.00 - Metric validation = 0.90.
Iteracion = 16
Metric train = 0.98 - Metric validation = 0.92.
I

In [21]:
grid_results_rf = grid_results.sort_values(by = ['metric_val', 'metric_train'], ascending = [False, False])
best_model_rf = grid_results_rf.iloc[0]
best_model_rf

max_features       5.000000
n_estimators    1000.000000
max_samples      100.000000
metric_train       0.984028
metric_val         0.924551
Name: 16, dtype: float64

In [22]:
# [3] define model
model = model_constructor(max_features = int(best_model_rf.max_features),
                          n_estimators = int(best_model_rf.n_estimators),
                          max_samples = int(best_model_rf.max_samples),
                          random_state = 0)

# [4] Train model
model.fit(dat[dat['dataset'] != 'test'].drop(['exitus', 'dataset'], axis = 1), dat[dat['dataset'] != 'test'].exitus.values)


# [5] Predict
pred_train = model.predict_proba(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1)) # predict!
pred_val = model.predict_proba(dat[dat['dataset'] == 'val'].drop(['exitus', 'dataset'], axis = 1)) # predict!
pred_test = model.predict_proba(dat[dat['dataset'] == 'test'].drop(['exitus', 'dataset'], axis = 1)) # predict!


# [6] Compute metric
metric_train = metric(dat[dat['dataset'] == 'train'].exitus.values, pred_train[:,1])
metric_val = metric(dat[dat['dataset'] == 'val'].exitus.values, pred_val[:,1])
metric_test = metric(dat[dat['dataset'] == 'test'].exitus.values, pred_test[:,1])

# print error
print('Metric train = %.2f - Metric val = %.2f - Metric test = %.2f'
      % (metric_train, metric_val, metric_test))

Metric train = 0.95 - Metric val = 0.94 - Metric test = 0.93


**- XGBoost**

In [23]:
from xgboost import XGBClassifier as model_constructor

In [24]:
# Xgboost
n_estimators_values = [1000]
learning_rate_values = [1]
gamma_values = [1, 10]
max_depth_values = [6, 20]
min_child_weight_values = [20, 100]
subsample_values = [0.1, 1]
colsample_bytree_values = [0.1, 1]
num_parallel_tree_values = [10]

params_grid = {'n_estimators': n_estimators_values,
                  'learning_rate': learning_rate_values,
                 'gamma': gamma_values,
                 'max_depth': max_depth_values,
                 'min_child_weight': min_child_weight_values,
                 'subsample': subsample_values,
                 'colsample_bytree': colsample_bytree_values,
                 'num_parallel_tree': num_parallel_tree_values}

In [25]:
num_iter = 1
grid_results = pd.DataFrame(columns = ('n_estimators',
                                       'learning_rate',
                                       'gamma',
                                       'max_depth',
                                       'min_child_weight',
                                       'subsample',
                                       'colsample_bytree',
                                       'num_parallel_tree',
                                       'best_iteration',
                                       'metric_train',
                                       'metric_val'))

for n_estimators in params_grid['n_estimators']:
    for learning_rate in params_grid['learning_rate']:
        for gamma in params_grid['gamma']:
            for max_depth in params_grid['max_depth']:
                for min_child_weight in params_grid['min_child_weight']:
                    for subsample in params_grid['subsample']:
                        for colsample_bytree in params_grid['colsample_bytree']:
                            for num_parallel_tree in params_grid['num_parallel_tree']:




                                                # Print trace
                                                print('Iteration = ' + str(num_iter))

                                                # [3] Define model
                                                model = model_constructor(n_estimators = n_estimators,
                                                                      learning_rate = learning_rate,
                                                                      gamma = gamma,
                                                                      max_depth = max_depth,
                                                                      min_child_weight = min_child_weight ,
                                                                      subsample = subsample,
                                                                      colsample_bytree = colsample_bytree,
                                                                      num_parallel_tree = num_parallel_tree,
                                                                      early_stopping_rounds = 10,
                                                                      eval_metric = "auc",
                                                                      random_state = 0) # nthread!!!

                                                # [4] Train model
                                                model.fit(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1), dat[dat['dataset'] == 'train'].exitus.values,
                                                          eval_set=[(dat[dat['dataset'] == 'val'].drop(['exitus', 'dataset'], axis = 1), dat[dat['dataset'] == 'val'].exitus.values)],
                                                          verbose = False)
                                                best_iteration = model.best_iteration

                                                # [5] Predict
                                                pred_train = model.predict_proba(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1)) # predict_proba!
                                                pred_val = model.predict_proba(dat[dat['dataset'] == 'val'].drop(['exitus', 'dataset'], axis = 1)) # predict_proba!

                                                # [6] Compute metric
                                                metric_train = metric(dat[dat['dataset'] == 'train'].exitus.values, pred_train[:, 1])
                                                metric_val = metric(dat[dat['dataset'] == 'val'].exitus.values, pred_val[:, 1])

                                                # print error
                                                print('AUC train = %.2f - AUC validation = %.2f.'
                                                      % (metric_train, metric_val))

                                                # Save iteration results
                                                grid_results.loc[num_iter]=[n_estimators,
                                                                            learning_rate,
                                                                            gamma,
                                                                            max_depth,
                                                                            min_child_weight,
                                                                            subsample,
                                                                            colsample_bytree,
                                                                            num_parallel_tree,
                                                                            best_iteration,
                                                                            metric_train,
                                                                            metric_val]
                                                num_iter += 1

Iteration = 1
AUC train = 0.50 - AUC validation = 0.50.
Iteration = 2
AUC train = 0.50 - AUC validation = 0.50.
Iteration = 3
AUC train = 0.97 - AUC validation = 0.92.
Iteration = 4
AUC train = 0.94 - AUC validation = 0.89.
Iteration = 5
AUC train = 0.50 - AUC validation = 0.50.
Iteration = 6
AUC train = 0.50 - AUC validation = 0.50.
Iteration = 7
AUC train = 0.50 - AUC validation = 0.50.
Iteration = 8
AUC train = 0.50 - AUC validation = 0.50.
Iteration = 9
AUC train = 0.50 - AUC validation = 0.50.
Iteration = 10
AUC train = 0.50 - AUC validation = 0.50.
Iteration = 11
AUC train = 0.97 - AUC validation = 0.92.
Iteration = 12
AUC train = 0.94 - AUC validation = 0.89.
Iteration = 13
AUC train = 0.50 - AUC validation = 0.50.
Iteration = 14
AUC train = 0.50 - AUC validation = 0.50.
Iteration = 15
AUC train = 0.50 - AUC validation = 0.50.
Iteration = 16
AUC train = 0.50 - AUC validation = 0.50.
Iteration = 17
AUC train = 0.50 - AUC validation = 0.50.
Iteration = 18
AUC train = 0.50 - AUC va

In [26]:
grid_results_xgb = grid_results.sort_values(by = ['metric_val', 'metric_train'], ascending = [False, False])
best_model_xgb = grid_results_xgb.iloc[0]
best_model_xgb

n_estimators         1000.000000
learning_rate           1.000000
gamma                   1.000000
max_depth               6.000000
min_child_weight       20.000000
subsample               1.000000
colsample_bytree        0.100000
num_parallel_tree      10.000000
best_iteration         37.000000
metric_train            0.970087
metric_val              0.920496
Name: 3, dtype: float64

In [27]:
# [3] define model
model = model_constructor(n_estimators = int(best_model_xgb.best_iteration),
                          learning_rate = best_model_xgb.learning_rate,
                          gamma = best_model_xgb.gamma,
                          max_depth = int(best_model_xgb.max_depth),
                          min_child_weight = best_model_xgb.min_child_weight,
                          subsample = best_model_xgb.subsample,
                          colsample_bytree = best_model_xgb.colsample_bytree,
                          num_parallel_tree = int(best_model_xgb.num_parallel_tree),
                          random_state = 0)

# [4] Train model
model.fit(dat[dat['dataset'] != 'test'].drop(['exitus', 'dataset'], axis = 1), dat[dat['dataset'] != 'test'].exitus.values)


# [5] Predict
pred_train = model.predict_proba(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1)) # predict!
pred_val = model.predict_proba(dat[dat['dataset'] == 'val'].drop(['exitus', 'dataset'], axis = 1)) # predict!
pred_test = model.predict_proba(dat[dat['dataset'] == 'test'].drop(['exitus', 'dataset'], axis = 1)) # predict!


# [6] Compute metric
metric_train = metric(dat[dat['dataset'] == 'train'].exitus.values, pred_train[:,1])
metric_val = metric(dat[dat['dataset'] == 'val'].exitus.values, pred_val[:,1])
metric_test = metric(dat[dat['dataset'] == 'test'].exitus.values, pred_test[:,1])

# print error
print('Metric train = %.2f - Metric val = %.2f - Metric test = %.2f'
      % (metric_train, metric_val, metric_test))

Metric train = 0.98 - Metric val = 0.95 - Metric test = 0.93


Compare the results obtained and answer the following questions:

**- Which model gives the best performance (over test)? Try to justify why this selection works.**

Both families of models gave almost identical results over the test. However, it is expected that XGBoost has more predictive potential, and its results could be improved by a more exhaustive grid search, as the one carried out here did not test a lot of hyperparameter combinations, which is criticial for XGBoost optimization.

**- Which model gives the worst performance (over test)? Try to justify why this selection does not work.**

Both families of models gave almost identical results over the test. However, it is expected that XGBoost has more predictive potential, and its results could be improved by a more exhaustive grid search, as the one carried out here did not test a lot of hyperparameter combinations, which is criticial for XGBoost optimization.

**- Which family of models seem to be most robust to hyperparameter selection based on the results obtained?**

In [28]:
grid_results_rf

,max_features,n_estimators,max_samples,metric_train,metric_val
16,5.0,1000.0,100.0,0.984028,0.924551
25,10.0,1000.0,100.0,0.981570,0.924036
22,10.0,100.0,100.0,0.980353,0.923331
7,2.0,1000.0,100.0,0.987116,0.920783
4,2.0,100.0,100.0,0.983456,0.916495
8,2.0,1000.0,1000.0,1.000000,0.914232
18,5.0,1000.0,1071.0,1.000000,0.913693
17,5.0,1000.0,1000.0,1.000000,0.913006
13,5.0,100.0,100.0,0.981216,0.912521
9,2.0,1000.0,1071.0,1.000000,0.911771


In [29]:
grid_results_xgb

,n_estimators,learning_rate,gamma,max_depth,min_child_weight,subsample,colsample_bytree,num_parallel_tree,best_iteration,metric_train,metric_val
3,1000.0,1.0,1.0,6.0,20.0,1.0,0.1,10.0,37.0,0.970087,0.920496
11,1000.0,1.0,1.0,20.0,20.0,1.0,0.1,10.0,37.0,0.970087,0.920496
19,1000.0,1.0,10.0,6.0,20.0,1.0,0.1,10.0,13.0,0.955336,0.916684
27,1000.0,1.0,10.0,20.0,20.0,1.0,0.1,10.0,13.0,0.955336,0.916684
20,1000.0,1.0,10.0,6.0,20.0,1.0,1.0,10.0,2.0,0.943731,0.893959
28,1000.0,1.0,10.0,20.0,20.0,1.0,1.0,10.0,2.0,0.943731,0.893959
4,1000.0,1.0,1.0,6.0,20.0,1.0,1.0,10.0,1.0,0.944875,0.892646
12,1000.0,1.0,1.0,20.0,20.0,1.0,1.0,10.0,1.0,0.944875,0.892646
1,1000.0,1.0,1.0,6.0,20.0,0.1,0.1,10.0,0.0,0.500000,0.500000
2,1000.0,1.0,1.0,6.0,20.0,0.1,1.0,10.0,0.0,0.500000,0.500000


Clearly Random Forest shows to be more robust to hyperparameters, with XGBoost creating some dummy models with AUC_train and AUC_val equal to 0.5.

# Additional models

As you ask me to do it, I have included here the necessary grid search code for the other families of models we have covered during the course, namely Decision Tree, SVM, and Neural Networks.

This was **NOT required** in the mock final exam, I have just added it here due to your request.

## Decision Tree

In [151]:
from sklearn.tree import DecisionTreeClassifier as model_constructor

In [31]:
criterion_values = ['gini', 'entropy']
max_depth_values = [None, 6, 20];
min_samples_split_values = [2, 5, 20];
min_samples_leaf_values = [1, 5, 20];
max_features_values = [None, 1, 2];

params_grid = {  'criterion': criterion_values,
                 'max_depth': max_depth_values,
                 'min_samples_split': min_samples_split_values,
                 'min_samples_leaf': min_samples_leaf_values,
                 'max_features': max_features_values}

In [32]:
num_iter = 1;
grid_results = pd.DataFrame(columns = ('criterion',
                                       'max_depth',
                                       'min_samples_split',
                                       'min_samples_leaf',
                                       'max_features',
                                       'metric_train',
                                       'metric_val'))

for criterion in params_grid['criterion']:
    for max_depth in params_grid['max_depth']:
        for min_samples_split in params_grid['min_samples_split']:
            for min_samples_leaf in params_grid['min_samples_leaf']:
                for max_features in params_grid['max_features']:

                    # Print trace
                    print('Iteration = ' + str(num_iter))

                    # [3] Define model
                    model = model_constructor(criterion = criterion,
                                              max_depth = max_depth,
                                              min_samples_split = min_samples_split,
                                              min_samples_leaf = min_samples_leaf,
                                              max_features = max_features,
                                              random_state = 0)

                    # [4] Train model
                    model.fit(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1), dat[dat['dataset'] == 'train'].exitus.values)

                    # [5] Predict
                    pred_train = model.predict_proba(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1)) # predict_proba!
                    pred_val = model.predict_proba(dat[dat['dataset'] == 'val'].drop(['exitus', 'dataset'], axis = 1)) # predict_proba!

                    # [6] Evaluate
                    metric_train = metric(dat[dat['dataset'] == 'train'].exitus.values, pred_train[:, 1])
                    metric_val = metric(dat[dat['dataset'] == 'val'].exitus.values, pred_val[:, 1])

                    # print error
                    print('AUC train = %.2f - AUC validation = %.2f.'
                          % (metric_train, metric_val))

                    # Save iteration results
                    grid_results.loc[num_iter]=[criterion,
                                                max_depth,
                                                min_samples_split,
                                                min_samples_leaf,
                                                max_features,
                                             metric_train,
                                             metric_val]
                    num_iter += 1

Iteration = 1
AUC train = 1.00 - AUC validation = 0.60.
Iteration = 2
AUC train = 1.00 - AUC validation = 0.56.
Iteration = 3
AUC train = 1.00 - AUC validation = 0.58.
Iteration = 4
AUC train = 0.99 - AUC validation = 0.73.
Iteration = 5
AUC train = 0.89 - AUC validation = 0.62.
Iteration = 6
AUC train = 0.98 - AUC validation = 0.72.
Iteration = 7
AUC train = 0.97 - AUC validation = 0.85.
Iteration = 8
AUC train = 0.70 - AUC validation = 0.60.
Iteration = 9
AUC train = 0.81 - AUC validation = 0.68.
Iteration = 10
AUC train = 1.00 - AUC validation = 0.64.
Iteration = 11
AUC train = 1.00 - AUC validation = 0.59.
Iteration = 12
AUC train = 1.00 - AUC validation = 0.62.
Iteration = 13
AUC train = 0.99 - AUC validation = 0.73.
Iteration = 14
AUC train = 0.89 - AUC validation = 0.62.
Iteration = 15
AUC train = 0.98 - AUC validation = 0.72.
Iteration = 16
AUC train = 0.97 - AUC validation = 0.85.
Iteration = 17
AUC train = 0.70 - AUC validation = 0.60.
Iteration = 18
AUC train = 0.81 - AUC va

In [33]:
grid_results_dt = grid_results.sort_values(by = ['metric_val', 'metric_train'], ascending = [False, False])
best_model_dt = grid_results_dt.iloc[0]
best_model_dt

criterion                gini
max_depth                None
min_samples_split           2
min_samples_leaf           20
max_features             None
metric_train         0.970164
metric_val           0.853077
Name: 7, dtype: object

In [153]:
# [3] define model
model = model_constructor(criterion = best_model_dt.criterion,
                          max_depth = best_model_dt.max_depth,
                          min_samples_split = int(best_model_dt.min_samples_split),
                          min_samples_leaf = int(best_model_dt.min_samples_leaf),
                          max_features = best_model_dt.max_features,
                          random_state = 0)

# [4] Train model
model.fit(dat[dat['dataset'] != 'test'].drop(['exitus', 'dataset'], axis = 1), dat[dat['dataset'] != 'test'].exitus.values)


# [5] Predict
pred_train = model.predict_proba(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1)) # predict!
pred_val = model.predict_proba(dat[dat['dataset'] == 'val'].drop(['exitus', 'dataset'], axis = 1)) # predict!
pred_test = model.predict_proba(dat[dat['dataset'] == 'test'].drop(['exitus', 'dataset'], axis = 1)) # predict!


# [6] Compute metric
metric_train = metric(dat[dat['dataset'] == 'train'].exitus.values, pred_train[:,1])
metric_val = metric(dat[dat['dataset'] == 'val'].exitus.values, pred_val[:,1])
metric_test = metric(dat[dat['dataset'] == 'test'].exitus.values, pred_test[:,1])

# print error
print('Metric train = %.2f - Metric val = %.2f - Metric test = %.2f'
      % (metric_train, metric_val, metric_test))

Metric train = 0.98 - Metric val = 0.97 - Metric test = 0.85


In [36]:
grid_results_dt

,criterion,max_depth,min_samples_split,min_samples_leaf,max_features,metric_train,metric_val
7,gini,None,2,20,None,0.970164,0.853077
16,gini,None,5,20,None,0.970164,0.853077
25,gini,None,20,20,None,0.970164,0.853077
34,gini,6,2,20,None,0.970164,0.853077
43,gini,6,5,20,None,0.970164,0.853077
...,...,...,...,...,...,...,...
146,entropy,20,5,1,1,0.998725,0.578694
56,gini,20,2,1,1,0.999995,0.575026
83,entropy,None,2,1,1,1.000000,0.558094
137,entropy,20,2,1,1,1.000000,0.558094


In this example, decision tree gives worse results than both Random Forest, and XGBoost, and is less robust than Random Forest regarding hyperparameter selection.

## SVM

In [33]:
from sklearn.svm import SVC as model_constructor
from sklearn.metrics import roc_auc_score as metric

In [28]:
# Get Cherksassky parameters --> This is optional!!!
d = dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1).shape[1];
m = np.mean(dat[dat['dataset']  == 'train'].exitus.values);
s = np.std(dat[dat['dataset']  == 'train'].exitus.values);

C_cherk = np.max([np.abs(m + 3*s),np.abs(m - 3*s)]);
gamma_cherk = np.power(0.2, 1/d)

In [41]:
# SVM
C_values = [0.3, 0.4, 0.5, 0.6]
gamma_values = [1e-03, 3e-03, gamma_cherk]

params_grid = {'C': C_values,
               'gamma': gamma_values}

In [42]:
num_iter = 1
grid_results = pd.DataFrame(columns = ('C',
                                       'gamma',
                                       'metric_train',
                                       'metric_val'))

for C in params_grid['C']:
    for gamma in params_grid['gamma']:

                    # Print trace
                    print('Iteration = ' + str(num_iter))

                    # [3] Define model
                    model = model_constructor(C = C,
                                              gamma = gamma,
                                              probability = True,
                                              random_state = 0) # Probability = True!!!

                    # [4] Train model
                    model.fit(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1), dat[dat['dataset']  == 'train'].exitus.values)

                    # [5] Predict
                    pred_train = model.predict_proba(dat[dat['dataset']  == 'train'].drop(['exitus', 'dataset'], axis = 1)) # predict_proba!
                    pred_val = model.predict_proba(dat[dat['dataset']  == 'val'].drop(['exitus', 'dataset'], axis = 1)) # predict_proba!

                    # [6] Compute metric
                    metric_train = metric(dat[dat['dataset']  == 'train'].exitus.values, pred_train[:,1])
                    metric_val = metric(dat[dat['dataset']  == 'val'].exitus.values, pred_val[:,1])

                    # print error
                    print('AUC train = %.2f - AUC validation = %.2f.'
                          % (metric_train, metric_val))

                    # Save iteration results
                    grid_results.loc[num_iter]=[C,
                                                gamma,
                                                metric_train,
                                                metric_val]
                    num_iter += 1

Iteration = 1
AUC train = 0.99 - AUC validation = 0.61.
Iteration = 2
AUC train = 1.00 - AUC validation = 0.63.
Iteration = 3
AUC train = 1.00 - AUC validation = 0.57.
Iteration = 4
AUC train = 0.99 - AUC validation = 0.61.
Iteration = 5
AUC train = 1.00 - AUC validation = 0.63.
Iteration = 6
AUC train = 1.00 - AUC validation = 0.57.
Iteration = 7
AUC train = 0.99 - AUC validation = 0.61.
Iteration = 8
AUC train = 1.00 - AUC validation = 0.63.
Iteration = 9
AUC train = 1.00 - AUC validation = 0.57.
Iteration = 10
AUC train = 0.99 - AUC validation = 0.62.
Iteration = 11
AUC train = 1.00 - AUC validation = 0.63.
Iteration = 12
AUC train = 1.00 - AUC validation = 0.57.


In [40]:
grid_results_svm = grid_results.sort_values(by = ['metric_val', 'metric_train'], ascending = [False, False])
best_model_svm = grid_results_svm.iloc[0]
best_model_svm

C               0.400000
gamma           0.003000
metric_train    0.997644
metric_val      0.634246
Name: 3, dtype: float64

In [36]:
# [3] Define model
model = model_constructor(C = best_model_svm.C,
                           gamma = best_model_svm.gamma,
                           probability = True,
                           random_state = 0) # Probability = True!!!

# [4] Train model
model.fit(dat[dat['dataset'] != 'test'].drop(['exitus', 'dataset'], axis = 1), dat[dat['dataset'] != 'test'].exitus.values)


# [5] Predict
pred_train = model.predict_proba(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1)) # predict!
pred_val = model.predict_proba(dat[dat['dataset'] == 'val'].drop(['exitus', 'dataset'], axis = 1)) # predict!
pred_test = model.predict_proba(dat[dat['dataset'] == 'test'].drop(['exitus', 'dataset'], axis = 1)) # predict!


# [6] Compute metric
metric_train = metric(dat[dat['dataset'] == 'train'].exitus.values, pred_train[:,1])
metric_val = metric(dat[dat['dataset'] == 'val'].exitus.values, pred_val[:,1])
metric_test = metric(dat[dat['dataset'] == 'test'].exitus.values, pred_test[:,1])

# print error
print('Metric train = %.2f - Metric val = %.2f - Metric test = %.2f'
      % (metric_train, metric_val, metric_test))

Metric train = 0.97 - Metric val = 0.99 - Metric test = 0.52


In [37]:
grid_results_svm

,C,gamma,metric_train,metric_val
1,0.500000,0.003000,0.997644,0.634197
3,0.999533,0.003000,0.997877,0.632919
4,0.999533,0.929456,1.000000,0.571061
2,0.500000,0.929456,1.000000,0.568197


In this example, SVM gives really bad results. This is probably due to a bad selection of hyperparameter values.

**Note:** Another possible factor for the bad results that we did not covered during the course, is that SVM way to calculate probabilities can be problematic, as it is totally based on distances and no real probabilites.

# Neural Networks

In [167]:
from sklearn.neural_network import MLPClassifier as model_constructor

In [105]:
# Neural networks
learning_rate_values = ['constant', 'adaptive']
learning_rate_init_values = [0.001, 0.1]
alpha_values = [0.0001, 0.001]
batch_size_values = [128, 256]
n_iter_no_change_values = [10]
activation_values = ['tanh']


hidden_layer_sizes = (100, 50, 25) # Not in the grid search due to format

params_grid = {'learning_rate': learning_rate_values,
                 'learning_rate_init': learning_rate_init_values,
                 'alpha': alpha_values,
                 'batch_size': batch_size_values,
                 'n_iter_no_change': n_iter_no_change_values,
                 'activation': activation_values}

In [159]:
num_iter = 1;
grid_results = pd.DataFrame(columns = ('learning_rate',
                                       'learning_rate_init',
                                       'alpha',
                                       'batch_size',
                                       'n_iter_no_change',
                                       'activation',
                                       'hidden_layer_sizes',
                                       'best_iteration',
                                       'metric_train',
                                       'metric_val'))

for learning_rate_init in params_grid['learning_rate_init']:
  for learning_rate in params_grid['learning_rate']:
    for alpha in params_grid['alpha']:
        for batch_size in params_grid['batch_size']:
            for n_iter_no_change in params_grid['n_iter_no_change']:
                for activation in params_grid['activation']:


                                                # Print trace
                                                print('Iteration = ' + str(num_iter))

                                                # [3] Define model
                                                model = model_constructor(learning_rate = learning_rate,
                                                                          learning_rate_init = learning_rate_init,
                                                                      hidden_layer_sizes = hidden_layer_sizes,
                                                                      alpha = alpha,
                                                                      batch_size = batch_size,
                                                                      n_iter_no_change = n_iter_no_change,
                                                                      activation = activation,
                                                                      early_stopping = True, # Important!!!
                                                                      random_state = 0)

                                                # [4] Train model
                                                model.fit(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1), dat[dat['dataset']  == 'train'].exitus.values)
                                                best_iteration = np.argmin(model.validation_scores_) + 1

                                                # [5] Predict
                                                pred_train = model.predict_proba(dat[dat['dataset']  == 'train'].drop(['exitus', 'dataset'], axis = 1)) # predict_proba!
                                                pred_val = model.predict_proba(dat[dat['dataset']  == 'val'].drop(['exitus', 'dataset'], axis = 1)) # predict_proba!

                                                # [6] Compute metric
                                                metric_train = metric(dat[dat['dataset']  == 'train'].exitus.values, pred_train[:,1])
                                                metric_val = metric(dat[dat['dataset']  == 'val'].exitus.values, pred_val[:,1])

                                                # print error
                                                print('AUC train = %.2f - AUC validation = %.2f.'
                                                      % (metric_train, metric_val))

                                                # Save iteration results
                                                grid_results.loc[num_iter]=[learning_rate,
                                                                            learning_rate_init,
                                                                            alpha,
                                                                            batch_size,
                                                                            n_iter_no_change,
                                                                            activation,
                                                                            hidden_layer_sizes,
                                                                            best_iteration,
                                                                            metric_train,
                                                                            metric_val]
                                                num_iter += 1

Iteration = 1
AUC train = 0.85 - AUC validation = 0.83.
Iteration = 2
AUC train = 0.31 - AUC validation = 0.32.
Iteration = 3
AUC train = 0.75 - AUC validation = 0.77.
Iteration = 4
AUC train = 0.31 - AUC validation = 0.31.
Iteration = 5
AUC train = 0.85 - AUC validation = 0.83.
Iteration = 6
AUC train = 0.31 - AUC validation = 0.32.
Iteration = 7
AUC train = 0.75 - AUC validation = 0.77.
Iteration = 8
AUC train = 0.31 - AUC validation = 0.31.
Iteration = 9
AUC train = 0.51 - AUC validation = 0.51.
Iteration = 10
AUC train = 0.35 - AUC validation = 0.37.
Iteration = 11
AUC train = 0.49 - AUC validation = 0.51.
Iteration = 12
AUC train = 0.17 - AUC validation = 0.23.
Iteration = 13
AUC train = 0.51 - AUC validation = 0.51.
Iteration = 14
AUC train = 0.35 - AUC validation = 0.37.
Iteration = 15
AUC train = 0.49 - AUC validation = 0.51.
Iteration = 16
AUC train = 0.17 - AUC validation = 0.23.


In [160]:
grid_results_nn= grid_results.sort_values(by = ['metric_val', 'metric_train'], ascending = [False, False])
best_model_nn = grid_results_nn.iloc[0]
best_model_nn

learning_rate              constant
learning_rate_init            0.001
alpha                        0.0001
batch_size                      128
n_iter_no_change                 10
activation                     tanh
hidden_layer_sizes    (100, 50, 25)
best_iteration                    1
metric_train               0.847641
metric_val                 0.826248
Name: 1, dtype: object

In [165]:
# [3] Define model
model = model_constructor(learning_rate = best_model_nn.learning_rate,
                          learning_rate_init = best_model_nn.learning_rate_init,
                          hidden_layer_sizes = best_model_nn.hidden_layer_sizes,
                          alpha = best_model_nn.alpha,
                          batch_size = best_model_nn.batch_size,
                          activation = best_model_nn.activation,
                          max_iter = best_model_nn.best_iteration,
                          early_stopping = False, # Important!!!
                          random_state = 0)

# [4] Train model
model.fit(dat[dat['dataset'] != 'test'].drop(['exitus', 'dataset'], axis = 1), dat[dat['dataset'] != 'test'].exitus.values)


# [5] Predict
pred_train = model.predict_proba(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1)) # predict!
pred_val = model.predict_proba(dat[dat['dataset'] == 'val'].drop(['exitus', 'dataset'], axis = 1)) # predict!
pred_test = model.predict_proba(dat[dat['dataset'] == 'test'].drop(['exitus', 'dataset'], axis = 1)) # predict!


# [6] Compute metric
metric_train = metric(dat[dat['dataset'] == 'train'].exitus.values, pred_train[:,1])
metric_val = metric(dat[dat['dataset'] == 'val'].exitus.values, pred_val[:,1])
metric_test = metric(dat[dat['dataset'] == 'test'].exitus.values, pred_test[:,1])

# print error
print('Metric train = %.2f - Metric val = %.2f - Metric test = %.2f'
      % (metric_train, metric_val, metric_test))

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(


Metric train = 0.70 - Metric val = 0.73 - Metric test = 0.76


In [166]:
grid_results_nn

,learning_rate,learning_rate_init,alpha,batch_size,n_iter_no_change,activation,hidden_layer_sizes,best_iteration,metric_train,metric_val
1,constant,0.001,0.0001,128,10,tanh,"(100, 50, 25)",1,0.847641,0.826248
5,adaptive,0.001,0.0001,128,10,tanh,"(100, 50, 25)",1,0.847641,0.826248
3,constant,0.001,0.0010,128,10,tanh,"(100, 50, 25)",1,0.747824,0.770443
7,adaptive,0.001,0.0010,128,10,tanh,"(100, 50, 25)",1,0.747824,0.770443
11,constant,0.100,0.0010,128,10,tanh,"(100, 50, 25)",1,0.491100,0.510169
15,adaptive,0.100,0.0010,128,10,tanh,"(100, 50, 25)",1,0.491100,0.510169
9,constant,0.100,0.0001,128,10,tanh,"(100, 50, 25)",1,0.509196,0.509746
13,adaptive,0.100,0.0001,128,10,tanh,"(100, 50, 25)",1,0.509196,0.509746
10,constant,0.100,0.0001,256,10,tanh,"(100, 50, 25)",3,0.346992,0.370642
14,adaptive,0.100,0.0001,256,10,tanh,"(100, 50, 25)",3,0.346992,0.370642


In this example, Neural Networks achieve better results than SVM, but still significantly worse than XGBoost and Random Forest, probably due to a suboptimal selection of hyperparameter values.

Moreover, results look extremely sensitive to hyperparameter selection, with the worse models obtaining AUC values over validation below 0.5.